In [108]:
import warnings
# warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"
warnings.filterwarnings('ignore')

In [109]:
import os
import pickle as pk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.optimizers import RMSprop
from keras.utils import np_utils

In [110]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import classification_report
import utils
from sklearn import preprocessing


In [111]:
import keras
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Conv1D, GlobalMaxPooling1D,Reshape,GlobalAveragePooling1D
from keras import Model, layers
from keras import Input
from keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout

In [112]:
def is_eq(a):
    return lambda b: 1 if a == b else 0

In [113]:
train = pd.read_csv(os.path.abspath('C:/Users/Admin/Desktop/work/IoT-device-type-identification-master/data/100_each_validation.csv'), low_memory=False)
validation = pd.read_csv(os.path.abspath('C:/Users/Admin/Desktop/work/IoT-device-type-identification-master/data/100_each_validation.csv'), low_memory=False)
test = pd.read_csv(os.path.abspath('C:/Users/Admin/Desktop/work/IoT-device-type-identification-master/data/100_each_test.csv'), low_memory=False)

In [114]:
 y_col = 'device_category'
# devs = train['device_category'].unique()
# devs[10]='unknown'
devs = ['security_camera', 'TV', 'smoke_detector', 'thermostat', 'water_sensor',
 'watch' ,'baby_monitor' ,'motion_sensor', 'lights' ,'socket','unknown']

In [115]:
x_train, y_train = utils.split_data(train,y_col)
x_test, y_test = utils.split_data(test,y_col)

y_train = np.asarray(y_train)
x_train = np.asarray(x_train)
y_test = np.asarray(y_test)
x_test = np.asarray(x_test)

lab_enc = preprocessing.LabelEncoder()
y_train = lab_enc.fit_transform(y_train)
y_test = lab_enc.fit_transform(y_test)
x_train = x_train.astype("float32")
y_train = y_train.astype("float32")
x_test = x_test.astype("float32")
y_test = y_test.astype("float32")

In [116]:

x_train = utils.perform_feature_selection(x_train, y_train, 240)
x_test = utils.perform_feature_selection(x_test, y_test, 240)
print(x_train.shape)

**********FEATURE SELECTION**********
[  1   2   4   5   6   9  10  11  12  13  14  15  16  17  18  19  20  21
  22  23  24  26  27  28  29  31  32  34  35  36  37  38  39  40  41  42
  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  59  60  61
  62  64  65  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 100 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 133 134 135 136 137
 138 139 140 141 143 144 145 146 147 149 150 151 152 153 154 155 156 157
 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 209 210 211 213 214 215 216 218 219 221 222 223 224 225 226
 227 229 230 231 232 233 234 235 236 237 239 240 241 242 247 248 249 250
 251 252 253 254 255 258 260 261 264 267 268 271 274 275 276 277 278 284
 285 286 290 

In [117]:
num_classes =10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
print('New y_train shape: ', y_train.shape)

New y_train shape:  (1000, 10)


In [118]:
TIME_PERIODS=80 #80
num_sensors=3 # 3
input_shape=240 # 240

model_m = Sequential()
model_m.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
model_m.add(Conv1D(100, 10, activation='relu', input_shape=(TIME_PERIODS, num_sensors)))
model_m.add(Conv1D(100, 10, activation='relu'))
model_m.add(MaxPooling1D(3))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(GlobalAveragePooling1D())
model_m.add(Dropout(0.5))
model_m.add(Dense(num_classes, activation='softmax'))
print(model_m.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 80, 3)             0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 71, 100)           3100      
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 62, 100)           100100    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 20, 100)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 11, 160)           160160    
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 2, 160)            256160    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 160)              

In [119]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]

In [120]:
model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

In [121]:
BATCH_SIZE = 400
EPOCHS = 50

In [122]:
history = model_m.fit(x_train,
                      y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

Epoch 1/50
2/2 [==============================] - 1s 290ms/step - loss: 2.2873 - accuracy: 0.1737 - val_loss: 2.2075 - val_accuracy: 0.2500
Epoch 2/50
2/2 [==============================] - 0s 174ms/step - loss: 2.1607 - accuracy: 0.3237 - val_loss: 1.9840 - val_accuracy: 0.3200
Epoch 3/50
2/2 [==============================] - 0s 174ms/step - loss: 1.9228 - accuracy: 0.3950 - val_loss: 1.6548 - val_accuracy: 0.5200
Epoch 4/50
2/2 [==============================] - 0s 170ms/step - loss: 1.6505 - accuracy: 0.4475 - val_loss: 1.3438 - val_accuracy: 0.4800
Epoch 5/50
2/2 [==============================] - 0s 183ms/step - loss: 1.4712 - accuracy: 0.4638 - val_loss: 1.1476 - val_accuracy: 0.5250
Epoch 6/50
2/2 [==============================] - 0s 170ms/step - loss: 1.3825 - accuracy: 0.4750 - val_loss: 1.0400 - val_accuracy: 0.5850
Epoch 7/50
2/2 [==============================] - 0s 187ms/step - loss: 1.1708 - accuracy: 0.5600 - val_loss: 0.9813 - val_accuracy: 0.6200
Epoch 8/50
2/2 [====

2/2 [==============================] - 0s 184ms/step - loss: 0.3705 - accuracy: 0.8475 - val_loss: 0.3010 - val_accuracy: 0.8250
Epoch 24/50
2/2 [==============================] - 0s 197ms/step - loss: 0.3489 - accuracy: 0.8413 - val_loss: 0.2845 - val_accuracy: 0.8400
Epoch 25/50
2/2 [==============================] - 0s 171ms/step - loss: 0.3389 - accuracy: 0.8438 - val_loss: 0.2803 - val_accuracy: 0.8450
Epoch 26/50
2/2 [==============================] - 0s 190ms/step - loss: 0.3175 - accuracy: 0.8562 - val_loss: 0.2674 - val_accuracy: 0.8850
Epoch 27/50
2/2 [==============================] - 0s 160ms/step - loss: 0.3037 - accuracy: 0.8662 - val_loss: 0.2729 - val_accuracy: 0.8500
Epoch 28/50
2/2 [==============================] - 0s 198ms/step - loss: 0.2887 - accuracy: 0.8637 - val_loss: 0.2605 - val_accuracy: 0.8550
Epoch 29/50
2/2 [==============================] - 0s 178ms/step - loss: 0.2891 - accuracy: 0.8687 - val_loss: 0.2486 - val_accuracy: 0.8600
Epoch 30/50
2/2 [========

2/2 [==============================] - 0s 186ms/step - loss: 0.1984 - accuracy: 0.8850 - val_loss: 0.2158 - val_accuracy: 0.8700
Epoch 47/50
2/2 [==============================] - 0s 179ms/step - loss: 0.1924 - accuracy: 0.9062 - val_loss: 0.2153 - val_accuracy: 0.8600
Epoch 48/50
2/2 [==============================] - 0s 210ms/step - loss: 0.1929 - accuracy: 0.8900 - val_loss: 0.2200 - val_accuracy: 0.8450
Epoch 49/50
2/2 [==============================] - 0s 174ms/step - loss: 0.1942 - accuracy: 0.8938 - val_loss: 0.2204 - val_accuracy: 0.8950
Epoch 50/50
2/2 [==============================] - 0s 179ms/step - loss: 0.1985 - accuracy: 0.8813 - val_loss: 0.2171 - val_accuracy: 0.8900


In [123]:
score = model_m.evaluate(x_test, y_test, verbose=1)
print("\nAccuracy on test data: %0.2f" % score[1])
print("\nLoss on test data: %0.2f" % score[0])
#confusion_matrix
y_pred_test = model_m.predict(x_test)
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_test = np.argmax(y_test, axis=1)
print(classification_report(max_y_test, max_y_pred_test))

[[3.1538634e-04 2.7034333e-04 3.3211557e-04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.2527596e-04 1.8022889e-04 2.4908668e-04 ... 0.0000000e+00
  1.0000000e+00 0.0000000e+00]
 [3.1538634e-04 2.7034333e-04 3.3211557e-04 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [9.9121418e-04 1.0813733e-03 8.3028892e-04 ... 0.0000000e+00
  1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 0.0000000e+00]]
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
32/32 [==============================] - 0s 8ms/step - loss: 5.2872 - accuracy: 0.3210

Accuracy on test data: 0.32

Loss on test data: 5.29
              precision    recall  f1-score   support

           0       0.15      0.83      0.25       100
           1       0.86      0